In [1]:
import torch
from PIL import Image
from moellava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from moellava.conversation import conv_templates, SeparatorStyle
from moellava.model.builder import load_pretrained_model
from moellava.utils import disable_torch_init
from moellava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria


disable_torch_init()

model_path = 'LanguageBind/MoE-LLaVA-StableLM-1.6B-4e'  # choose a model
device = 'cuda'
load_4bit, load_8bit = False, False
model_name = get_model_name_from_path(model_path)
tokenizer, model, processor, context_len = load_pretrained_model(model_path, None, model_name, load_8bit, load_4bit, device=device)

/root/anaconda3/envs/moellava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-16 12:08:52,226] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/anaconda3/envs/moellava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-05-16 12:17:25,163] [INFO] [logging.py:96:log_dist] [Rank -1] Creating MoE layer with num_experts: 4 | num_local_experts: 4 | expert_parallel_size: 1
[2024-05-16 12:17:25,167] [INFO] [logging.py:96:log_dist] [Rank -1] Creating MoE layer with num_experts: 4 | num_local_experts: 4 | expert_parallel_size: 1
[2024-05-16 12:17:25,170] [INFO] [logging.py:96:log_dist] [Rank -1] Creating MoE layer with num_experts: 4 | num_local_experts: 4 | expert_parallel_size: 1
[2024-05-16 12:17:25,172] [INFO] [logging.py:96:log_dist] [Rank -1] Creating MoE layer with num_experts: 4 | num_local_experts: 4 | expert_parallel_size: 1
[2024-05-16 12:17:25,175] [INFO] [logging.py:96:log_dist] [Rank -1] Creating MoE layer with num_experts: 4 | num_local_experts: 4 | expert_parallel_size: 1
[2024-05-16 12:17:25,178] [INFO] [logging.py:96:log_dist] [Rank -1] Creating MoE layer with num_experts: 4 | num_local_experts: 4 | expert_parallel_size: 1
[2024-05-16 12:17:25,180] [INFO] [logging.py:96:log_dist] [Rank 

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]


[2024-05-16 12:17:29,151] [WARNING] [comm.py:152:init_deepspeed_backend] NCCL backend in DeepSpeed not yet implemented
[2024-05-16 12:17:29,152] [INFO] [comm.py:594:init_distributed] cdb=None
[2024-05-16 12:17:29,152] [INFO] [comm.py:609:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-05-16 12:17:29,173] [INFO] [comm.py:659:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.23.81.17, master_port=29500
[2024-05-16 12:17:29,174] [INFO] [comm.py:625:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-05-16 12:17:29,176] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.9.5, git-hash=unknown, git-branch=unknown
[2024-05-16 12:17:29,177] [INFO] [logging.py:96:log_dist] [Rank 0] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


/root/anaconda3/envs/moellava/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
import os

image_processor = processor['image']
conv_mode = "phi"  # phi or qwen or stablelm
conv = conv_templates[conv_mode].copy()
print(conv)
print(type(conv))
roles = conv.roles
print(roles)

# 获取当前已分配的显存量
allocated_memory = 0
print(f"当前已分配显存量：{torch.cuda.memory_allocated()-allocated_memory} 字节")
allocated_memory = torch.cuda.memory_allocated()



root_path = 'moellava/serve/examples'
img_pth = os.listdir(root_path)
img_list = [os.path.join(root_path,img) for img in img_pth]

image = 'moellava/serve/examples/WechatIMG4.jpg'
inp = 'Describe the image.'
# inp_list = [inp for i in range(len(img_list))]





print(f"{roles[1]}: {inp}")
inp = DEFAULT_IMAGE_TOKEN + '\n' + inp

conv.append_message(conv.roles[0], inp)
conv.append_message(conv.roles[1], None)
print(conv.messages)
prompt = conv.get_prompt()

input_id = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
# input_id_list = [input_id for _ in range(len(img_list))]
# input_ids = torch.cat(input_id_list,0)

# print(input_ids.shape)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2

keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_id)
print(type(stopping_criteria))



# print(f"当前已分配显存量：{torch.cuda.memory_allocated()-allocated_memory} 字节")
# allocated_memory = torch.cuda.memory_allocated()




Conversation(system="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", roles=('USER', 'ASSISTANT'), messages=[], offset=0, sep_style=<SeparatorStyle.TWO: 2>, sep=' ', sep2='<|endoftext|>', version='phi', skip_next=False)
<class 'moellava.conversation.Conversation'>
('USER', 'ASSISTANT')
当前已分配显存量：6406559232 字节
ASSISTANT: Describe the image.
[['USER', '<image>\nDescribe the image.'], ['ASSISTANT', None]]
<class 'moellava.mm_utils.KeywordsStoppingCriteria'>


In [24]:

def inference(image):
    image_tensor = image_processor.preprocess(Image.open(image).convert('RGB'), return_tensors='pt')['pixel_values'].to(model.device, dtype=torch.float16)
    # img_tensor_list = [image_processor.preprocess(Image.open(image).convert('RGB'), return_tensors='pt')['pixel_values'].to(model.device, dtype=torch.float16) for image in img_list]
    # img_tensors = torch.cat(img_tensor_list)
    # print(img_tensors.shape)
    with torch.inference_mode():
        output_ids = model.generate(
            input_id,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=1024,
            use_cache=True,
            stopping_criteria=[stopping_criteria],
            pad_token_id = 100257,
            eos_token_id = 100257
            )

    return output_ids


In [26]:
import time 
start_time = time.time()

for image in img_list:

    output_ids = inference(image)
    outputs = tokenizer.decode(output_ids[0, input_id.shape[1]:], skip_special_tokens=True).strip()
    print(outputs)
# print(input_id.shape)
# print(output_ids.shape)


end_time = time.time()
sum_time = end_time - start_time
avg_time = sum_time/len(img_list)
print(f"每张照片花费平均时间为{avg_time}s")
print(f"当前已分配显存量：{torch.cuda.memory_allocated()-allocated_memory} 字节")
allocated_memory = torch.cuda.memory_allocated()

6
The image depicts a beautiful beach scene with a city in the background. The city is lit up at night, creating a stunning contrast between the vibrant lights and the serene ocean. The beach is filled with palm trees, and the water appears to be calm and inviting. The overall atmosphere of the scene is picturesque and inviting, showcasing the beauty of both the city and the beach.
The image features a black and white cat sitting on a person's arm. The cat appears to be looking at the camera, and its eyes are wide open. The scene takes place indoors, with the person's arm visible in the foreground. The cat seems to be enjoying the attention and affection from its owner.
The image shows a man standing on the back of a yellow taxi cab, using a clothes iron to press clothes. The scene takes place on a busy city street, with other vehicles such as a car and a bus visible in the background. The man is wearing a yellow shirt and appears to be focused on his task.
The image features a bed wit

In [14]:
import requests
import base64
def get_dispatch_req(dev):
    if dev:
        url = 'http://test-infra-internal.conss.co/api/ms/v1/infra-internal/aigc/model-dispatch'

    else:
        url = 'http://infra-internal-us.conss.co/api/ms/v1/infra-internal/aigc/model-dispatch'

    return url


def req_caption(img_base64, url):

    gen_params = {
                "uid": "1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9",
                "dispatch_model_type": 6,
                "dispatch_proxy_mode": 2,
                "params": {
                    "img": ''
                }
    }

    error_msg = None

    try:
        gen_params['params']['img'] = img_base64
        res = requests.post(url,
        json=gen_params,
        stream=True,
        timeout=10)  # Send the HTTP request with a timeout of 10 seconds
        res.raise_for_status()  # Raise an exception if the response status code is not 2xx

        return res.json(), error_msg

    except requests.exceptions.Timeout as e:
        return None, str(e)

    except requests.exceptions.RequestException:
        # Handle other request errors (e.g., service does not exist)

        return None, str(requests.exceptions.RequestException)

def get_img_caption(image_path):
        dev = True
        url = get_dispatch_req(dev)
        # image_path = '/mnt/MoE-LLaVA/moellava/serve/examples/extreme_ironing.jpg'
        with open(image_path, "rb") as img_file:
            img_base64 = base64.b64encode(img_file.read()).decode('utf-8')

        print(req_caption(img_base64, url)[0])

        
start_time = time.time()
for img in img_list:
    
    get_img_caption(img)
end_time = time.time()
avg_time = -(start_time - end_time)/len(img_list)
print(avg_time)
print(f"当前已分配显存量：{torch.cuda.memory_allocated()-allocated_memory} 字节")
allocated_memory = torch.cuda.memory_allocated()

{'params': {'caption': 'a city at sunset with a large body of water'}, 'uid': '1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9'}
{'params': {'caption': "a black and white cat laying on a person's lap"}, 'uid': '1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9'}
{'params': {'caption': 'a woman riding a scooter down a city street'}, 'uid': '1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9'}
{'params': {'caption': 'a television screen with a clock on top of it'}, 'uid': '1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9'}
{'params': {'caption': 'two elephants standing next to each other on a beach'}, 'uid': '1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9'}
{'params': {'caption': 'a wooden dock on a body of water'}, 'uid': '1005_182821815599579138_182823065434980352_5e664d90-1069-4c69-9470-02d481f61bf9'}
1.5796031951904297
当前已分配显存量：0 字

In [27]:
import sys
print(sys.path)


['/root/anaconda3/envs/moellava/lib/python310.zip', '/root/anaconda3/envs/moellava/lib/python3.10', '/root/anaconda3/envs/moellava/lib/python3.10/lib-dynload', '', '/root/anaconda3/envs/moellava/lib/python3.10/site-packages', '__editable__.moellava-1.0.0.finder.__path_hook__', '/tmp/tmp8hpushlx']
